# MNIST in TF 

#### Dependencies 

In [1]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

C:\Users\moi\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### Load data


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('MNIST_data/',  one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


####  hyperparameters

In [3]:
lr = 0.1
epochs=20
batch_size=128
weight_initializer=tf.contrib.layers.xavier_initializer()

In [4]:
n_input=784
n_dense_1=64
n_dense_2=64
n_classes=10

#### Placeholder

In [5]:
x=tf.placeholder(tf.float32, [None, n_input]) 

In [6]:
y=tf.placeholder(tf.float32, [None, n_classes])

####  Layers

In [7]:
def dense(x,W,b):
    z= tf.add(tf.matmul(x,W), b)
    # ponieważ mamy relu activation, to pass z into relu activation function, to get output a
    a=tf.nn.relu(z)
    return a

#### Architecture 

In [8]:
def network(x,weights, biases):
    # 2 dense layers
    dense_1=dense(x, weights['W1'], biases['b1'])
    dense_2=dense(dense_1, weights['W2'], biases['b2'])
    out_layer_z = tf.add(tf.matmul(dense_2, weights['W_out']), biases['b_out']) 
    return out_layer_z

#### Variable dict

In [9]:
bias_dict={
   'b1':tf.Variable(tf.zeros([n_dense_1])), # a 1-dimensional array, let's be zeros, wcześniej zdeifniowane: ma 64 neurons 
    'b2':tf.Variable(tf.zeros([n_dense_2])),
    'b_out': tf.Variable(tf.zeros([n_classes]))
}

In [10]:
weights_dict={
    'W1':tf.get_variable('W1', [n_input, n_dense_1], initializer=weight_initializer),  # 1. parameter - name of variable,  
                                  # 2. parameter:2-dim. matrix, it has to have right shape (multipl. with x)
    'W2':tf.get_variable('W2', [n_dense_1, n_dense_2], initializer=weight_initializer),
    'W_out':tf.get_variable('W_out', [n_dense_2, n_classes], initializer=weight_initializer)
}

#### Model 

In [11]:
predictions=network(x, weights=weights_dict, biases=bias_dict)

In [12]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
    # reducing of the array value to a single scalar;
    # 2 values: prediction and the true labels y
optimizer=tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(cost)
    #minimizing the error using GradientDescentOptimizer

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



#### Metrics 

In [13]:
correct_prediction=tf.equal(tf.argmax(predictions, 1), tf.argmax(y,1))

In [14]:
accuracy_pct=tf.reduce_mean(tf.cast(correct_prediction, tf.float32))*100


#### Initialization 

In [15]:
initializer_op=tf.global_variables_initializer()

#### Training 

In [16]:
with tf.Session() as session:  #create an instance of the session
    session.run(initializer_op) # initialize variable in the session (running init._op)
    
    print('Training for', epochs, 'epochs.')
    
    #loop over epochs:
    
    for epoch in range(epochs):
        avg_cost=0.0
        avg_accuracy_pct=0.0
        
        
        #how many batches: 
        n_batches=int(mnist.train.num_examples / batch_size)
        
        for i in range (n_batches):
            batch_x, batch_y=mnist.train.next_batch(batch_size)
            # feeding batch to run a single step of SGD 
            # and fetch the cost & accuracy
            
            _, batch_cost, batch_acc=session.run([optimizer, cost, accuracy_pct], 
                                                 feed_dict={x:batch_x, y:batch_y})
            
            #accumulate and calculate the avg. cost/accu over all batches within the epoch
            
            avg_cost+=batch_cost/n_batches
            avg_accuracy_pct+=batch_acc/n_batches
            
            
        # print logs about progress, at the end of epoch:
        print('Epoch', '%3d' %(epoch+1),
                  ': cost = ', '{:.3f}'.format(avg_cost),
                  ': accuracy =', '{:.2f}'.format(avg_accuracy_pct), '%', 
                  sep='')
            
    print('Testing model.\n')
            
    test_cost=cost.eval({x:mnist.test.images, y:mnist.test.labels})
    test_accuracy_pct=accuracy_pct.eval({x:mnist.test.images, y:mnist.test.labels})
            
    print('Test Cost:', '{:.3f}'.format(test_cost))
    print('Test Accuracy: ', '{:.2f}'.format(test_accuracy_pct), '%', sep='')

        

Training for 20 epochs.
Epoch  1: cost = 0.501: accuracy =85.40%
Epoch  2: cost = 0.251: accuracy =92.71%
Epoch  3: cost = 0.194: accuracy =94.35%
Epoch  4: cost = 0.159: accuracy =95.38%
Epoch  5: cost = 0.134: accuracy =96.08%
Epoch  6: cost = 0.117: accuracy =96.59%
Epoch  7: cost = 0.104: accuracy =96.95%
Epoch  8: cost = 0.093: accuracy =97.20%
Epoch  9: cost = 0.083: accuracy =97.53%
Epoch 10: cost = 0.077: accuracy =97.72%
Epoch 11: cost = 0.070: accuracy =97.92%
Epoch 12: cost = 0.064: accuracy =98.11%
Epoch 13: cost = 0.059: accuracy =98.23%
Epoch 14: cost = 0.055: accuracy =98.37%
Epoch 15: cost = 0.051: accuracy =98.50%
Epoch 16: cost = 0.047: accuracy =98.63%
Epoch 17: cost = 0.044: accuracy =98.69%
Epoch 18: cost = 0.040: accuracy =98.88%
Epoch 19: cost = 0.037: accuracy =98.97%
Epoch 20: cost = 0.035: accuracy =99.03%
Testing model.

Test Cost: 0.085
Test Accuracy: 97.18%
